In [10]:
import sys
import os
import re
import numpy as np
import torch
from torch.multiprocessing import Process, set_start_method
from functools import partial
from utils import record_stats, display_stats, distribute
from pathlib import Path 


%load_ext autoreload
%autoreload 2

In [19]:
n_cpu = 12
n_instance = 12
nodesels = ['estimate_estimate', 'estimate_dummy', 'expert_dummy','gnn_dummy', 'svm_dummy']

problem = "FCMCNF"
normalize = True
device = 'cuda' if torch.cuda.is_available() else 'cpu'
verbose = True
on_log = False
default = True
delete = True

if delete:
    try:
        import shutil
        shutil.rmtree(os.path.join(os.path.abspath(''), 
                                       f'stats/{problem}'))
    except:
        ''



instances = list(Path(os.path.join(os.path.abspath(''), 
                                   f"./problem_generation/data/{problem}/test")).glob("*.lp"))
if n_instance == -1 :
    n_instance = len(instances)

instances = instances[:n_instance]

print("Evaluation")
print(f"  Problem:                    {problem}")
print(f"  n_instance/problem:         {len(instances)}")
print(f"  Nodeselectors evaluated:    {','.join( ['default' if default else '' ] + nodesels)}")
print(f"  Device for GNN inference:   {device}")
print(f"  Normalize features:         {normalize}")
print("----------------")




Evaluation
  Problem:                    FCMCNF
  n_instance/problem:         12
  Nodeselectors evaluated:    default,estimate_estimate,estimate_dummy,expert_dummy,gnn_dummy,svm_dummy
  Device for GNN inference:   cuda
  Normalize features:         True
----------------


In [16]:

processes = [  Process(name=f"worker {p}", 
                                target=partial(record_stats,
                                                nodesels=nodesels,
                                                instances=instances[p1:p2], 
                                                problem=problem,
                                                device=torch.device(device),
                                                normalize=normalize,
                                                verbose=verbose,
                                                default=default))
                for p,(p1,p2) in enumerate(distribute(n_instance, n_cpu)) ]  


try:
    set_start_method('spawn')
except RuntimeError:
    ''

a = list(map(lambda p: p.start(), processes)) #run processes
b = list(map(lambda p: p.join(), processes)) #join processes

min_n = min([ int( str(instance).split('=')[2 if problem in ['FCMCNF'] else 1].split('_')[0] )  for instance in instances ] )
max_n = max([ int( str(instance).split('=')[2 if problem in ['FCMCNF'] else 1].split('_')[0] )  for instance in instances ] )

display_stats(problem, nodesels, instances, min_n, max_n, default=default)

<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
<All keys matched successfully>
Statistics on FCMCNF over 12 instances for problem size in [80, 98]
  default, n=12 
    Mean NNode Created   : 599.00
    Mean Solving Time    : 16.91
-------------------------------------------------
  oracle, n=12 
    Mean NNode Created   : 449.08
    Mean Solving Time    : 16.80
-------------------------------------------------
  gnn_trained, n=12 
    Mean NNode Created   : 862.25
    Mean Solving Time    : 36.56
     |---   Feature Extraction  Mean Time:      7.85
     |---   Feature Normalization Mean Time:    0.25
     |---   Inference Mean Time:                4.04
     |---   Ncomp Mean:           